In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.metrics import adjusted_mutual_info_score
from sklearn.model_selection import GridSearchCV
import joblib

In [2]:
%%time
df = pd.read_csv("../data/df_CatVal_cleanedV3.csv",
               encoding="utf-8", low_memory=False, index_col=0)

CPU times: total: 39.4 s
Wall time: 39.5 s


In [3]:
df.shape

(1833614, 144)

In [4]:
df = df.sample(200000)

In [5]:
df.dropna(subset=['product_name'], inplace=True)

In [6]:
df_init = df

In [7]:
df['pnns1'] = pd.factorize(df['pnns1'])[0]

### Normalisation

In [8]:
# Sélectionner les colonnes numériques
numeric_cols = list(df.select_dtypes(include=["float64","int64"]).columns)

In [9]:
# Créer un scaler StandardScaler
scaler = StandardScaler()

In [10]:
# Appliquer la normalisation aux colonnes numériques
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [11]:
# Sélectionner les colonnes numériques
cols_to_convert_int = list(df.select_dtypes(include=["int64"]).columns)
cols_to_convert_float = list(df.select_dtypes(include=["float64"]).columns)

# Changing format of object and float64 columns
df[cols_to_convert_int] = df[cols_to_convert_int].astype('int32')
df[cols_to_convert_float] = df[cols_to_convert_float].astype('float32')

In [12]:
# Supprimer les colonnes numériques non utiles à l'apprentissage
numeric_cols.remove('pnns1')
numeric_cols.remove('nutriscore')
numeric_cols.remove('unique_scans_n')
numeric_cols.remove('completeness')

### X et y

In [ ]:
X = df[numeric_cols]

In [ ]:
y = df['product_name']

### Petite PCA

In [13]:
# Réduire la dimensionnalité avec PCA
pca = PCA(n_components=20)
pca_df = pd.DataFrame(pca.fit_transform(df[numeric_cols]))

## DBSCAN

In [23]:
labels_true = df['pnns1']
X = pca_df

In [24]:
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

def dbscan_clusters(X, eps=0.3, min_samples=10):
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print("Estimated number of clusters: %d" % n_clusters_)
    print("Estimated number of noise points: %d" % n_noise_)
    print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
    print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
    print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
    print("Adjusted Rand Index: %0.3f" % metrics.adjusted_rand_score(labels_true, labels))
    print(
        "Adjusted Mutual Information: %0.3f"
        % metrics.adjusted_mutual_info_score(labels_true, labels)
    )
    print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(X, labels))

    # #############################################################################
    # Plot result
    import matplotlib.pyplot as plt

    # Black removed and is used for noise instead.
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = labels == k

        xy = X[class_member_mask & core_samples_mask]
        plt.plot(
            xy[:, 0],
            xy[:, 1],
            "o",
            markerfacecolor=tuple(col),
            markeredgecolor="k",
            markersize=14,
        )

        xy = X[class_member_mask & ~core_samples_mask]
        plt.plot(
            xy[:, 0],
            xy[:, 1],
            "o",
            markerfacecolor=tuple(col),
            markeredgecolor="k",
            markersize=6,
        )

    plt.title("Estimated number of clusters: %d" % n_clusters_)
    plt.show()

In [25]:
dbscan_clusters(pca_df, eps=0.35, min_samples=20)

Estimated number of clusters: 233
Estimated number of noise points: 71568
Homogeneity: 0.501
Completeness: 0.398
V-measure: 0.444
Adjusted Rand Index: 0.386


C:\Users\joell\VSCode\OpenFoodFact_Cluster\env\lib\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
C:\Users\joell\VSCode\OpenFoodFact_Cluster\env\lib\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
C:\Users\joell\VSCode\OpenFoodFact_Cluster\env\lib\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
C:\Users\joell\VSCode\OpenFoodFact_Cluster\env\lib\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received cont

Adjusted Mutual Information: 0.442
Silhouette Coefficient: -0.372


InvalidIndexError: (slice(None, None, None), 0)